In [1]:
import pandas as pd
data = pd.read_csv('./phone_params_tags.csv')

In [2]:
data.columns

Index(['CPU型号', 'CPU频率', 'GPU型号', 'RAM容量', 'ROM容量', 'id', '主屏分辨率', '主屏尺寸',
       '价格', '前置摄像头', '后置摄像头', '型号', '摄像头总数', '核心数', '电池容量', 'tags', 'cpu'],
      dtype='object')

In [8]:
import re
import numpy as np

In [61]:
rom_p = re.compile('(\d+)[G|T]B')
def extract_rom(s):
    if type(s) == float:
        return np.nan
    match = re.match(rom_p,s)
    if match:
        return match.group(1)
    else:
        return np.nan

roms = data['ROM容量'].apply(extract_rom)

In [62]:
ram_p = re.compile('(\d+)GB')
def extract_ram(s):
    if type(s) == float:
        return np.nan
    match = re.match(rom_p,s)
    if match:
        return match.group(1)
    else:
        return np.nan

rams = data['RAM容量'].apply(extract_ram)

In [12]:
def extract_price(s):
    if type(s) == float:
        return np.nan
    try:
        s = int(s)
        return s
    except ValueError:
        return np.nan

price = data['价格'].apply(extract_price)

In [28]:
camera_p = re.compile(r'[\dx]*[双|三|四]*\d+万像素')
def extract_camera(s):
    if type(s) == float:
        return np.nan
    match = re.findall(camera_p,s)
    if len(match) == 0:
        print(s)
        return np.nan
    return '+'.join(match)

camera_front = data['前置摄像头'].apply(extract_camera)
camera_back = data['后置摄像头'].apply(extract_camera)

1600高清级像素高像素自拍手机>
800普通级像素高像素自拍手机>
2400高清级像素高像素自拍手机>
1600高清级像素高像素自拍手机>
800普通级像素高像素自拍手机>
1600高清级像素高像素自拍手机>
800普通级像素高像素自拍手机>
1600高清级像素高像素自拍手机>
800普通级像素高像素自拍手机>
500普通级像素高像素自拍手机>
1300像素高清级像素高像素自拍手机>
30普通级像素高清像素手机>
30万万像素普通级像素高清像素手机>
30万万像素普通级像素高清像素手机>


In [43]:
num_p = re.compile(r'(\d)x(\d+)万像素')
num_p2 = re.compile(r'(\d+)万像素')
def sum_camera(s):
    if type(s) == float:
        return np.nan
    s = s.split('+')
    total = 0
    for pixel in s:
        if 'x' in pixel:
            match = re.search(num_p,pixel)
            total += int(match.group(1)*int(match.group(1)))
        elif '双' in s:
            match = re.search(num_p2,pixel)
            total += 2*int(match.group(1))
        else:
            match = re.search(num_p2,pixel)
            total += int(match.group(1))
    return total

pixel_front = camera_front.apply(sum_camera)
pixel_back = camera_back.apply(sum_camera)

In [50]:
brand_list = '''
OPPO vivo 华为 荣耀 三星 苹果 一加 努比亚 魅族 联想 金立 中兴 Moto 锤子科技 360 国美手机 小米 夏普 华硕 美图 诺基亚 HTC 8848
SUGAR 黑莓 海信 AGM 黑鲨 索尼 谷歌 LG 酷派 VERTU 中国移动 飞利浦 联想 ZUK 小辣椒 TCL 天语 YotaPhone 长虹 MANNROG 微软
格力 朵唯 纽曼 雷蛇 大神 传音 ivvi 海尔 酷比 索野 康佳 誉品 乐目 邦华 COMIO 青橙 创星 卡布奇诺 独影天幕 詹姆士 21克 汇威 百合
波导 守护宝 ioutdoor 保千里 私人医生 阿尔卡特 朗界 E人E本 红鸟 sonim PPTV 尼凯恩 innos 云狐 新石器 柯达 会播 富可视 VEB
铂爵 青想 米蓝 传奇 途为 imoo 神舟 BDV TP-LINK 易百年 小格雷 首云 克里特 先锋 图灵 小宇宙 泛泰 大唐 电信 雅马亚虎 VANO VAIO
松下 东芝 惠普 全普 奥克斯 欧恩
'''
brand_list = brand_list.split()
brand_list = [brand.strip() for brand in brand_list]
brand_list = [brand for brand in brand_list if brand]

In [60]:
def extract_brand(s):
    if type(s) == float:
        return np.nan
    for brand in brand_list:
        if brand in s:
            return brand
    return np.nan

In [54]:
size_p = re.compile(r'(\d\.\d+)英寸')
def extract_size(s):
    if type(s) == float:
        return np.nan
    match = re.match(size_p,s)
    if match:
        return float(match.group(1))
    else:
        return np.nan
size = data['主屏尺寸'].apply(extract_size)

In [55]:
data.columns

Index(['CPU型号', 'CPU频率', 'GPU型号', 'RAM容量', 'ROM容量', 'id', '主屏分辨率', '主屏尺寸',
       '价格', '前置摄像头', '后置摄像头', '型号', '摄像头总数', '核心数', '电池容量', 'tags', 'cpu'],
      dtype='object')

In [63]:
sql_data = {'cpu':data['cpu'],'memory':rams,'disk':roms,'id':data['id'],'size':size,'price':price,'camera_front':camera_front,
           'camera_back':camera_back,'name':data['型号'],'brand':brand,'tags':data['tags'],
           'pixel_front':pixel_front,'pixel_back':pixel_back}
sql_data = pd.DataFrame(sql_data)

In [64]:
from sqlalchemy import Column, String, create_engine
from sqlalchemy.orm import sessionmaker

#%%
engine = create_engine('mysql+mysqlconnector://root@localhost:3306/dialog')
#%%
engine.execute('drop table if exists phone_param')
sql_data.to_sql('phone_param',engine,if_exists = 'replace',chunksize=1000)